In [1]:
# pkgs = [
# 	"CategoricalArrays","Chain","Conda","CSV","DataFrames","Dates",
# 	"FileIO","HTTP","ImageShow","LaTeXStrings","Metrics","Plots",
# 	"Plotly","PrettyTables","PyCall","Statistics","StatsBase",
# 	"StatsPlots","StateSpaceModels","XLSX"
# 	];
# expr = "using " * join(pkgs, ", ")
# eval(Meta.parse(expr))
# import Pkg
# Pkg.add(pkgs)

In [2]:
#=using PlutoUI; PlutoUI.TableOfContents(
    title = "Pronóstico de Inflación por Componentes", 
    depth = 5)=#
#using CategoricalArrays,Chain,Conda,CSV,DataFrames,Dates,FileIO,HTTP
#using ImageShow,LaTeXStrings,Metrics,Plots,Plotly,PrettyTables,PyCall
#using Statistics,StatsBase,StatsPlots,StateSpaceModels,XLSX;
using Chain,Conda,CSV,DataFrames,Dates,FileIO,GLM
using Plots,PrettyTables,PyCall,StateSpaceModels,StatsPlots
Plots.plotly()
# Pkg.add(PackageSpec(name="PyCall", rev="master"))
# Pkg.build("PyCall")

# within a Julia REPL
# using Conda
# Conda.pip_interop(true)
# Conda.pip("install", "webio_jupyter_extension")

wd = @__DIR__

#get_grouped_indices()
include(wd * "/Functions/update_data.jl");  # Obtener información
include(wd * "/Functions/plot_type.jl");   # Gráficos dinámicos o estáticos
include(wd * "/Functions/apply_models.jl");   # Modelos	

\pagebreak

# Resumen Ejecutivo

En este documento se expone un proceso en Julia para calcular variaciones mensuales e interanuales (y la contribución de los componentes a dichas variaciones) a partir de los componentes del Indice de Precios al Consumidor (IPC). En el mismo se obtiene lo siguiente, por cada agrupación y sus componentes:

- Índice;
- Tasa de crecimiento mensual (pendiente);
- Tasa de crecimiento interanual (pendiente);
- Contribución a la tasa de crecimiento mensual (pendiente); y
- Contribución a la tasa de crecimiento interanual (pendiente).

El procedimiento se realiza mediante funciones programadas, por lo que los cálculos a partir de los grupos se realizan automáticamente, tomando en cuenta el número de grupos que se necesite crear; la automatización implica la eliminación de errores que puedan generarse al usar fórmulas en Excel.

\pagebreak

# Datos Iniciales

## Página Web, grupos


In [3]:
# Conda.pip("install", "openpyxl")
import Conda
# Conda.add("openpyxl")
# Conda.add("pandas")
#using PyCall
#PyCall.@pyimport openpyxl
pd = PyCall.pyimport("pandas");

In [4]:
# Read Excel data
url = "https://www.bch.hn/estadisticos/GIE/LIBSERIE%20IPC%20RUBROS/Serie%20Mensual%20y%20Promedio%20Anual%20del%20%C3%8Dndice%20de%20Precios%20al%20Consumidor%20por%20Rubros.xlsx"
df_cpi_grupos = pd.read_excel(url)

# Drop columns and rows with missing values
df_cpi_grupos = df_cpi_grupos.dropna(axis=1, thresh=3)
df_cpi_grupos = df_cpi_grupos.dropna(axis=0, thresh=3)
df_cpi_grupos = df_cpi_grupos.dropna(axis=1)

# Drop first row (name of variables in Spanish)
df_cpi_grupos = df_cpi_grupos.drop(df_cpi_grupos.index[0])

# Rename columns
df_cpi_grupos.columns = [
    "Fechas",
    "Alimentos y Bebidas no Alcohólicas",
    "Bebidas Alcohólicas y Tabaco",
    "Prendas de Vestir y Calzado",
    "Alojamiento, Agua, Electricidad, Gas y Otros Combustibles",
    "Muebles y Artículos para la Conservación del Hogar",
    "Salud",
    "Transporte",
    "Comunicaciones",
    "Recreación y Cultura",
    "Educación",
    "Restaurantes y Hoteles",
    "Cuidado Personal",
    "IPC"]

# To DataFrame
df_cpi_grupos = DataFrames.DataFrame([col => Base.collect(df_cpi_grupos[col]) for col in df_cpi_grupos.columns])
df_cpi_grupos = df_cpi_grupos[2:end,:]

# Drop rows containing "PROMEDIO"
df_cpi_grupos = DataFrames.filter(
	row -> row.Fechas != "PROMEDIO", df_cpi_grupos)

# # Add months as date
df_cpi_grupos.Fechas = Base.collect(
	Dates.Date(1991,1,1):Dates.Month(1):Dates.Date(1991,1,1)+Dates.Month(size(df_cpi_grupos)[1]-1))

# Guardar en archivo y leer en formato correcto
CSV.write(
	wd * "/Results/data_rubros_web.csv", 
	delim = ';',
	df_cpi_grupos);
df_cpi_grupos = CSV.read(
	#dir * "./Data/Base/data.csv",
	wd * "/Results/data_rubros_web.csv", 
	DataFrames.DataFrame);

En este informe se presenta el gráfico del primer componente del grupo, sin embargo, puede obtenerse un gráfico por cada componente y del IPC general^[j=13 en el código.].


In [5]:
tipo = "line";
grupos2 = DataFrames.DataFrame(
	No = 1:Base.size(df_cpi_grupos, 2)-1,
	Nombre = DataFrames.names(df_cpi_grupos)[2:end]);
nombrar = Base.Array(grupos2.Nombre);
nombrar = string.(nombrar);
j = 2#size(nombrar)[1] # 1:13

#dynamic_plot(df, nombrar[i], tipo)
# static_plot(df_cpi_grupos, nombrar[j], tipo)
df, col_name, plot_type = df_cpi_grupos, nombrar[j], tipo
col_name = Symbol(col_name)
plot_type = Symbol(plot_type)
fig_001 = Plots.plot(
    df.Fechas, 
    df[:, col_name],
    legend = false,
    label = String(col_name),
    title = String(col_name),)
#=Plots.savefig(
	fig_001,
	wd * "/Figuras/fig_001.png")
FileIO.load(wd * "/Figuras/fig_001.png")=#

Los pesos de cada rubro pueden aproximarse mediante una regresión lineal:


In [6]:
data = copy(df_cpi_grupos[:,2:end])
xnames = ["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","Y"]
DataFrames.rename!(data, Base.names(data) .=> xnames);
reg = GLM.lm(GLM.@formula(Y ~  0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12),data)
w = coef(reg)
data[:,:IPC_estim] = data[:,end]
data[!,:IPC_estim] = 
	data.x1 .* w[1] + data.x2 .* w[2] + data.x3 .* w[3] + 
	data.x4 .* w[4] + data.x5 .* w[5] + data.x6 .* w[6] +
	data.x7 .* w[7] + data.x8 .* w[8] + data.x9 .* w[9] + 
	data.x10 .* w[10] + data.x11 .* w[11] + data.x12 .* w[12]
DataFrames.rename!(
	data, 
	Base.names(data) .=> 
	vcat(Base.names(df_cpi_grupos)[2:end],"IPC_Estim"));
data = hcat(df_cpi_grupos[:,1],data)
DataFrames.rename!(
	data, 
	Base.names(data)[1] .=> 
	"Fechas");
formato_tabla(data[end-20:end,1:6])
formato_tabla(data[end-20:end,7:end])

 ------------ ------------------------------------ ------------------------------ ----------------------------- ----------------------------------------------------------- ----------------------------------------------------
      Fechas   Alimentos y Bebidas no Alcohólicas   Bebidas Alcohólicas y Tabaco   Prendas de Vestir y Calzado   Alojamiento, Agua, Electricidad, Gas y Otros Combustibles   Muebles y Artículos para la Conservación del Hogar 
        Date                              Float64                        Float64                       Float64                                                     Float64                                              Float64 
 ------------ ------------------------------------ ------------------------------ ----------------------------- ----------------------------------------------------------- ----------------------------------------------------
  2022-02-01                             320.3000                       459.1000                    

 ---------- ------------ ---------------- ---------------------- ----------- ------------------------ ------------------ ---------- -----------
     Salud   Transporte   Comunicaciones   Recreación y Cultura   Educación   Restaurantes y Hoteles   Cuidado Personal        IPC   IPC_Estim 
   Float64      Float64          Float64                Float64     Float64                  Float64            Float64    Float64     Float64 
 ---------- ------------ ---------------- ---------------------- ----------- ------------------------ ------------------ ---------- -----------
  477.0000     409.2000          77.4000               272.6000    744.2000                 402.2000           348.9000   374.3000    374.3416
  480.7000     413.5000          77.3000               277.6000    744.2000                 404.5000           350.9000   377.8000    378.0593
  481.4000     445.9000          77.3000               278.9000    744.2000                 409.6000           353.3000   384.2000    384.

# Pronóstico, Rubros

## Modelos univariados


In [7]:
df_for = DataFrame()
Threads.@threads for i in 1:size(df_cpi_grupos)[2]-1
	# i = 1
	x = Vector(df_cpi_grupos[!,i+1])
	seas = 12
	steps_ahead = 12 # Número de meses a pronosticar
	Fechas = Base.collect(
		df_cpi_grupos[end,1] + Dates.Month(1):
		Dates.Month(1):
		df_cpi_grupos[end,1] + Dates.Month(steps_ahead))
	df = univariate_models(x, seas, steps_ahead)

	df = hcat(Fechas,df)
	DataFrames.rename!(
		df, 
		Base.names(df) .=> 
		vcat("Fechas",DataFrames.names(df)[2:end]));
	df = DataFrames.stack(df,2:size(df)[2])
	df = @chain df begin
		DataFrames.rename(
		:variable => :Serie,
		:value => :Indice)
	end
	df.Variable .= names(df_cpi_grupos)[i+1]
	append!(df_for,df)
	return df_for
end
df_for

┌ Warning: The optimization process converged but the Hessian matrix is not positive definite. This means that StateSpaceModels.jl cannot estimate the distribution of the hyperparameters If you are interested in estimates of the distribution of ther hyperparameters we advise you tochange the optimization algorithm by using the kwarg fit(...; optimizer = Optimizer(StateSpaceModels.Optim.THE_METHOD_OF_YOUR_CHOICE()))The list of possible algorithms can be found on this link https://julianlsolvers.github.io/Optim.jl/stable/# otherwise you can simply skip this proccess by using fit(...; save_hyperparameter_distribution=false) 
└ @ StateSpaceModels C:\Users\ec109004\.julia\packages\StateSpaceModels\DmxMT\src\fit.jl:57


Row,Fechas,Serie,Indice,Variable
,Date,String,Float64,String
1,2023-11-01,sarima,387.778,Alimentos y Bebidas no Alcohólicas
2,2023-12-01,sarima,389.422,Alimentos y Bebidas no Alcohólicas
3,2024-01-01,sarima,390.137,Alimentos y Bebidas no Alcohólicas
4,2024-02-01,sarima,391.817,Alimentos y Bebidas no Alcohólicas
5,2024-03-01,sarima,393.12,Alimentos y Bebidas no Alcohólicas
6,2024-04-01,sarima,394.645,Alimentos y Bebidas no Alcohólicas
7,2024-05-01,sarima,395.347,Alimentos y Bebidas no Alcohólicas
8,2024-06-01,sarima,397.202,Alimentos y Bebidas no Alcohólicas
9,2024-07-01,sarima,399.644,Alimentos y Bebidas no Alcohólicas


In [8]:
df_cpi = DataFrames.stack(df_cpi_grupos,2:size(df_cpi_grupos)[2])
df_cpi = @chain df_cpi begin
	DataFrames.rename(
		:variable => :Variable,
		:value => :Indice)
end
df_cpi.Serie .= "Observada"
df_all = vcat(df_for,df_cpi)
CSV.write(
	wd * "/Results/Pronostico_Grupos.csv", 
	delim = ";",
	df_all)

"C:\\Directorio_Trabajo\\GitHub\\IPC_Honduras-main/Results/Pronostico_Grupos.csv"

Graficar pronósticos de una serie:


In [9]:
Plots.plotly()
df_plot = DataFrames.filter(
	:Variable => ==("Alimentos y Bebidas no Alcohólicas"),
	df_all)
df_plot = DataFrames.filter(
	:Fechas => >=(Dates.Date.("2022-01-01")),
	df_plot)
@df df_plot Plots.plot(
	:Fechas, :Indice, 
	group = :Serie, 
	legend=:topleft)

# Página Web, Regiones


In [10]:
# url = "https://www.bch.hn/estadisticos/GIE/LIBSerie%20IPC%20Region/Serie%20Mensual%20y%20Promedio%20Anual%20del%20%C3%8Dndice%20de%20Precios%20al%20Consumidor%20por%20Regi%C3%B3n.xlsx"

# # Read Excel data
# df_cpi_regiones = pd.read_excel(url)

# # Drop columns and rows with missing values
# df_cpi_regiones = df_cpi_regiones.dropna(axis=1, thresh=3)
# df_cpi_regiones = df_cpi_regiones.dropna(axis=0, thresh=3)
# df_cpi_regiones = df_cpi_regiones.dropna(axis=1)

# # Drop first row (name of variables in Spanish)
# df_cpi_regiones = df_cpi_regiones.drop(df_cpi_regiones.index[0])

# # Rename columns
# df_cpi_regiones.columns = [
#     "Fechas",
#     "Central_Metropolitana",
#     "Resto_Central",
#     "Norte_Metropolitana",
#     "Resto_Norte",
#     "Sur",
#     "Oriental",
#     "Occidental",
#     "IPC"];

# # To DataFrame
# df_cpi_regiones = DataFrames.DataFrame([col => Base.collect(df_cpi_regiones[col]) for col in df_cpi_regiones.columns])
# df_cpi_regiones = df_cpi_regiones[2:end,:]

# # # Drop rows containing "PROMEDIO"
# df_cpi_regiones = DataFrames.filter(
# 	row -> row.Fechas != "Promedio", df_cpi_regiones)

# # Add months as date
# df_cpi_regiones.Fechas = Base.collect(Dates.Date(1991,1,1):Dates.Month(1):Dates.Date(1991,1,1)+Dates.Month(size(df_cpi_regiones)[1]-1))
# # df_cpi_regiones = df_cpi_regiones[:,vcat(size(df_cpi_regiones)[2],collect(2:size(df_cpi_regiones)[2]-1))]

# # Guardar en archivo y leer en formato correcto
# CSV.write(
# 	wd * "/Results/data_regiones_web.csv", 
# 	delim = ';',
# 	df_cpi_regiones);
# df_cpi_regiones = CSV.read(
# 	#dir * "./Data/Base/data.csv",
# 	wd * "/Results/data_regiones_web.csv", 
# 	DataFrames.DataFrame);

In [11]:
# tipo = "line";
# grupos2 = DataFrames.DataFrame(
# 	No = 1:Base.size(df_cpi_regiones, 2)-1,
# 	Nombre = DataFrames.names(df_cpi_regiones)[2:end]);
# nombrar = Base.Array(grupos2.Nombre);
# nombrar = string.(nombrar);
# j = 1#size(nombrar)[1] # 1:8

# #dynamic_plot(df, nombrar[i], tipo)
# # static_plot(df_cpi_grupos, nombrar[j], tipo)
# df, col_name, plot_type = df_cpi_regiones, nombrar[j], tipo
# col_name = Symbol(col_name)
# plot_type = Symbol(plot_type)
# fig_002 = Plots.plot(
#     df.Fechas, 
#     df[:, col_name],
#     legend = false,
#     label = String(col_name),
#     title = String(col_name),)
# #=Plots.savefig(
# 	fig_002,
# 	wd * "/Figuras/fig_002.png")
# FileIO.load(wd * "/Figuras/fig_002.png")=#

También puede aplicarse una regresión lineal para estimar los pesos por región:


In [12]:
# data = copy(df_cpi_regiones[:,2:end])
# reg = GLM.lm(GLM.@formula(IPC ~  0 + Central_Metropolitana + Resto_Central + Norte_Metropolitana + Resto_Norte + Sur + Oriental + Occidental),data)
# w = coef(reg)
# data[:,:IPC_estim] = data[:,end]
# data[!,:IPC_estim] = 
# 	data.Central_Metropolitana .* w[1] + data.Resto_Central .* w[2] + data.Norte_Metropolitana .* w[3] + 
# 	data.Resto_Norte .* w[4] + data.Sur .* w[5] + data.Oriental .* w[6] +
# 	data.Occidental .* w[7]
# data = hcat(df_cpi_regiones[:,1],data)
# DataFrames.rename!(
# 	data, 
# 	Base.names(data)[1] .=> 
# 	"Fechas");
# formato_tabla(data[end-20:end,:])